In [ ]:
import os
import pandas as pd
from tqdm import tqdm

In [ ]:
matches = pd.read_csv("Football/football-matches-1872-to-2018.csv")
print(len(matches))
matches.head(1)

In [ ]:
matches["league"].value_counts()

In [ ]:
match_map = {"FIFA World Cup": "World: FIFA World Cup",
             "UEFA Euro": "Europe: Euro", 
             "FIFA Confederations Cup": "World: FIFA Confederations Cup",
             "Copa America Centenario": "South America: Copa America"}

## Combining data of players representing a team in each competition by year

In [ ]:
squad = []
for i in os.listdir("Football/squad_member/"):
    print(i.split('_')[0], " ".join(i.split('_')[1:-1]))
    if len(squad) == 0:
        squad = pd.read_csv("Football/squad_member/"+i)
        squad["year"] = [i.split('_')[0]]*len(squad)
        squad["league"] = [match_map[" ".join(i.split('_')[1:-1])]]*len(squad)
        squad["first_name"] = squad["player_name"].apply(lambda x: x.split()[0])
        squad["last_name"] = squad["player_name"].apply(lambda x: x.split()[-1])
    else:
        temp = pd.read_csv("Football/squad_member/"+i)
        temp["year"] = [i.split('_')[0]]*len(temp)
        temp["league"] = [match_map[" ".join(i.split('_')[1:-1])]]*len(temp)
        temp["first_name"] = temp["player_name"].apply(lambda x: x.split()[0])
        temp["last_name"] = temp["player_name"].apply(lambda x: x.split()[-1])
        squad = squad.append(temp)
    print(len(squad))
squad.head(5)

In [ ]:
squad.to_csv("Football/all_squads.csv")

## Combining data of EA Sports Fifa player ratings for every year's release of the game

In [ ]:
players = []
for i in os.listdir("Football/player/data clean/"):
    print("20"+i.split('_')[0][-2:])
    if len(players) == 0:
        players = pd.read_csv("Football/player/data clean/"+i)
        players["year"] = ["20"+i.split('_')[0][-2:]]*len(players)
        players["first_name"] = players["Name"].apply(lambda x: x.split()[0])
        players["last_name"] = players["Name"].apply(lambda x: x.split()[-1])
    else:
        temp = pd.read_csv("Football/player/data clean/"+i)
        temp["year"] = ["20"+i.split('_')[0][-2:]]*len(temp)
        temp["first_name"] = temp["Name"].apply(lambda x: x.split()[0])
        temp["last_name"] = temp["Name"].apply(lambda x: x.split()[-1])
        players = players.append(temp)
    print(len(players))
players = players[["Name","Country","Club","OverRate","PotRate","Height","Weight","PreferredFoot","BirthDate","Age","year","first_name","last_name"]]
players.head(1)

In [ ]:
players.to_csv("Football/all_players.csv")

## Merging Squad and Player rating data

In [ ]:
squad = pd.read_csv("Football/all_squads.csv")
players = pd.read_csv("Football/all_players.csv")

ratings = []
age = []
for n, row in squad.iterrows():
    if "Serbia" in row["Country"]:
        row["Country"] = "Serbia"
    if "Trinidad" in row["Country"]:
        row["Country"] = "Trinidad & Tobago"
    temp = players[(players["Country"] == row["Country"]) & (players["year"] == row["year"])]
    temp = temp[temp["first_name"] == row["first_name"]]
    if len(temp) > 1:
        temp = temp[temp["last_name"] == row["last_name"]]
    if len(temp):
        ratings.append(temp["OverRate"].to_list()[0])
        age.append(temp["Age"].to_list()[0])
    else:
        print(row["year"], row["player_name"], row["Country"])
        ratings.append(None)
        age.append(None)
#     break

In [ ]:
ratings.count(None)

In [ ]:
squad["ratings"] = ratings
squad["age"] = age
squad.head(1)

In [ ]:
squad["ratings"] = squad["ratings"].fillna(squad["ratings"].min())
squad["age"] = squad["age"].fillna(int(squad["age"].mean()))

squad.to_csv("Football/squad_withratings.csv")

In [ ]:
def dates(x):
    x = x.split()[0]
    if '-' in x:
        return x.split('-')[-1]
    elif '/' in x:
        return x.split('/')[-1]
    elif '\\' in x:
        return x.split('\\')[-1]
    else:
        print(x)

In [ ]:
flag = {}
for i in list(match_map.values()):
    flag[i] = True
flag

## Formatting data to ensure the competitions are uniform across the dataset

In [ ]:
map_to_league = {"World: Friendly International": "No",
"World: FIFA World Cup Qualification": "World: FIFA World Cup",
"Europe: Euro Qualification": "Europe: Euro",
"Africa: Africa Cup of Nations Qualification": "World: FIFA World Cup",
"World: FIFA World Cup": "World: FIFA World Cup",
"Asia: Asian Cup Qualification": "World: FIFA World Cup",
"Africa: Africa Cup of Nations": "World: FIFA World Cup",
"North&South America: Gold Cup": "No",
"Africa: African Nations Championship": "No",
"Europe: Euro": "Europe: Euro",
"South America: Copa America": "South America: Copa America",
"Asia: Gulf Cup": "No",
"Asia: AFF Suzuki Cup": "No",
"Asia: Asian Cup": "World: FIFA World Cup",
"Africa: African Nations Championship Qualification": "No",
"World: FIFA Confederations Cup": "World: FIFA Confederations Cup",
"Asia: AFF Suzuki Cup Qualification": "No",
"Europe: Nations Cup": "Europe: Euro",
"North&South America: Gold Cup Qualification": "No",
"South America: Copa America Qualification": "South America: Copa America"}

## Assigning average Player rating and average Age for each team in a match

In [ ]:
matches = pd.read_csv("Football/football-matches-1872-to-2018.csv")
matches["year"] = matches["match_date"].apply(lambda x: int(dates(x)))
matches = matches[matches["year"] >= 2006]
matches["league"] = matches["league"].apply(lambda x: map_to_league[x])
matches = matches[[flag.get(i, False) for i in matches["league"]]]
matches = matches[matches["team_1"].apply(lambda x: "U19" not in x)]
matches = matches[matches["team_1"].apply(lambda x: "U20" not in x)]
print(len(matches))
matches.head(1)

In [ ]:
matches["league"].value_counts()

In [ ]:
result = []
for n, row in matches.iterrows():
    if row["score_1"] == row["score_2"]:
        result.append("Draw")
    elif row["score_1"] > row["score_2"]:
        result.append("Team1_Win")
    else:
        result.append("Team2_Win")
matches["result"] = result
matches.head(1)

In [ ]:
squad = pd.read_csv("Football/squad_withratings.csv")
squad = squad.replace("Serbia and Montenegro", "Serbia")
squad = squad.replace("Trinidad and Tobago", "Trinidad & Tobago")
squad.head(1)

In [ ]:
avgRat_team1 = []
avgRat_team2 = []
avgAge_team1 = []
avgAge_team2 = []
for n, row in matches.iterrows():
    if "Serbia" in row["team_1"]:
        row["team_1"] = "Serbia"
    if "Serbia" in row["team_2"]:
        row["team_2"] = "Serbia"
    temp1 = squad[(squad["Country"] == row["team_1"]) & (squad["year"] == int(row["year"])) & (squad["league"] == row["league"])]
    if len(temp1) == 0:
        print(row["year"], row["team_1"])
        avgRat_team1.append(None)
        avgAge_team1.append(None)
    else:
        avgRat_team1.append(temp1["ratings"].mean())
        avgAge_team1.append(temp1["age"].mean())
    temp2 = squad[(squad["Country"] == row["team_2"]) & (squad["year"] == int(row["year"])) & (squad["league"] == row["league"])]
    if len(temp2) == 0:
        print(row["year"], row["team_2"])
        avgRat_team2.append(None)
        avgAge_team2.append(None)
    else:
        avgRat_team2.append(temp2["ratings"].mean())
        avgAge_team2.append(temp2["age"].mean())

In [ ]:
matches["avgRat_team1"] = avgRat_team1
matches["avgRat_team2"] = avgRat_team2
matches["avgAge_team1"] = avgAge_team1
matches["avgAge_team2"] = avgAge_team2
matches.head(1)

In [ ]:
matches = matches[["league", "match_date", "team_1", "team_2", "score_1", "score_2", "result", "year", "avgRat_team1","avgRat_team2","avgAge_team1","avgAge_team2"]]
print(len(matches))
matches["avgRat_team1"] = matches["avgRat_team1"].fillna(matches["avgRat_team1"].min())
matches["avgRat_team2"] = matches["avgRat_team2"].fillna(matches["avgRat_team2"].min())
matches["avgAge_team1"] = matches["avgAge_team1"].fillna(matches["avgAge_team1"].mean())
matches["avgAge_team2"] = matches["avgAge_team2"].fillna(matches["avgAge_team2"].mean())
matches.head(1)

In [ ]:
matches.to_csv("Football/matches_withratings.csv")

## Encorporating Country data for every given year: GDP, Population, Avg. years of schooling

In [ ]:
res = {"Country": [], "year": [], "Population": [], "GDP": [], "avg_years_of_schooling": []}
df = pd.read_csv("Countries/GDP.csv")
df_pop = pd.read_csv("Countries/Population.csv")
df_pop["Country Name"] = df_pop["Country Name"].replace("Egypt, Arab Rep.", "Egypt")
df_pop["Country Name"] = df_pop["Country Name"].replace("Iran, Islamic Rep.", "Iran")

df_school = pd.read_csv("Countries/GDL-Mean-years-schooling-data.csv")
df_school["Country"] = df_school["Country"].replace("Argentina urban", "Argentina")
df_school["Country"] = df_school["Country"].replace("Chili", "Chile")
df_school["Country"] = df_school["Country"].replace("Trinidad & Tobago", "Trinidad and Tobago")
df_school["Country"] = df_school["Country"].replace("Czech Republic", "Czechia")
df_school["Country"] = df_school["Country"].replace("Slovakia", "Slovak Republic")
df_school["Country"] = df_school["Country"].replace("Turkey", "Turkiye")
df_school["Country"] = df_school["Country"].replace("Venezuela", "Venezuela, RB")
df_school["Country"] = df_school["Country"].replace("South Korea", "Korea, Rep.")


for n, row in df.iterrows():
    if "Egypt" in row["Country Name"]:
        row["Country Name"] = "Egypt"
    if "Iran" in row["Country Name"]:
        row["Country Name"] = "Iran"
    temp_pop = df_pop[df_pop["Country Name"] == row["Country Name"]]
#     temp_dens = df_dens[df_dens["Country Name"] == row["Country Name"]]
    temp_school = df_school[df_school["Country"] == row["Country Name"]]
    if len(temp_school) == 0:
        print(row["Country Name"])
        continue
    for y in range(2006,2018+1):
#         temp_dens_byYear = temp_dens[temp_dens["Year"] == y]
#         print(temp_pop[str(y)])
        temp_pop_byYear = float(temp_pop[str(y)])
        temp_school_byYear = float(temp_school[str(y)])
#         print(row["Country Name"], temp_pop, y)
        res["Country"].append(row["Country Name"])
        res["year"].append(y)
        res["Population"].append(temp_pop_byYear)
        res["GDP"].append(row[str(y)])
        res["avg_years_of_schooling"].append(temp_school_byYear)
#         res["Population_Density"].append(temp_dens_byYear["Count"].to_list()[0])
#     break
res = pd.DataFrame(res)
print(len(res))
res.head()

In [ ]:
res = res.replace("United Kingdom", "England")
res = res.replace("Trinidad and Tobago", "Trinidad & Tobago")
res = res.replace("Serbia", "Serbia and Montenegro")
res = res.replace("Czechia","Czech Republic")
res = res.replace("Russian Federation", "Russia")
res = res.replace("Slovak Republic", "Slovakia")
res = res.replace("Turkiye","Turkey")
res = res.replace("United States", "USA")
res = res.replace("Venezuela, RB", "Venezuela")
res = res.replace("Korea, Rep.", "South Korea")

In [ ]:
res.to_csv("Countries/combined.csv")

## Mergining all of the data to get a set of matches with additional data for each game

In [ ]:
matches = pd.read_csv("Football/matches_withratings.csv", index_col=[0])
matches = matches.replace("Serbia", "Serbia and Montenegro")
matches.head(1)

In [ ]:
res = {"team1_Population": [], "team1_GDP": [], "team1_avg_years_of_schooling": [], "team2_Population": [], "team2_GDP": [], "team2_avg_years_of_schooling": []}
country_data = pd.read_csv("Countries/combined.csv")


for n, row in matches.iterrows():
    temp1 = country_data[(country_data["Country"] == row["team_1"]) & (country_data["year"] == row["year"])]
    if len(temp1) != 1:
#         print(row["team_1"],row["year"])
        res["team1_Population"].append(None)
        res["team1_GDP"].append(None)
        res["team1_avg_years_of_schooling"].append(None)
    else:
        res["team1_Population"].append(temp1["Population"].to_list()[0])
        res["team1_GDP"].append(temp1["GDP"].to_list()[0])
        res["team1_avg_years_of_schooling"].append(temp1["avg_years_of_schooling"].to_list()[0])
        
    temp2 = country_data[(country_data["Country"] == row["team_2"]) & (country_data["year"] == row["year"])]
    if len(temp2) != 1:
#         print(row["team_2"],row["year"])
        res["team2_Population"].append(None)
        res["team2_GDP"].append(None)
        res["team2_avg_years_of_schooling"].append(None)
    else:
        res["team2_Population"].append(temp2["Population"].to_list()[0])
        res["team2_GDP"].append(temp2["GDP"].to_list()[0])
        res["team2_avg_years_of_schooling"].append(temp2["avg_years_of_schooling"].to_list()[0])
        
matches["team1_Population"] = res["team1_Population"]
matches["team1_GDP"] = res["team1_GDP"]
matches["team1_avg_years_of_schooling"] = res["team1_avg_years_of_schooling"]
matches["team2_Population"] = res["team2_Population"]
matches["team2_GDP"] = res["team2_GDP"]
matches["team2_avg_years_of_schooling"] = res["team2_avg_years_of_schooling"]
len(matches)

In [ ]:
matches.head(1)

In [ ]:
matches.to_csv("Final_Data.csv")